# Ejercicio 2 - Implementación de Red Neuronal

Este ejercicio se centra en el entrenamiento y evaluación de una red neuronal (relativamente) simple, sobre un conjunto de datos de campañas de marketing de productos bancarios. 

## Contexto: Análisis de éxito en campaña de marketing

Fuente: https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

El foco está en la implementación de una red neuronal para clasificación y la comparación de su desempeño con otro modelo de clasificación para predicir el valor de un atributo, desde un *dataset* de información de un resultados de personas contactadas por una campaña de marketing y que compraron la oferta (atributo "OK"), con cerca de 41.200 registros de personas contactadas.

Este conjunto de datos (abierto para este tipo de usos instruccionales), consiste en 20 atributos y 1 clase de etiquetas (totalizando 21 columnas) y corresponde a los datos de una campaña telefónica a diversos clientes en Portugal, ofreciéndoles la compra de un producto bancario. En varios casos, un cliente fue contactado varias veces antes de aceptar el el depósito a plazo ofrecido por la campaña (OK = yes). 

Algunos de los atributos relevantes son (combinando atributos categóricos, con numéricos):
* **Datos personales**: Edad, Ocupación, Estado Civil, Nivel de Educación.
* **Datos financieros**: Su casa tiene crédito hipotecario, default: si el crédito ha caído en quiebra; tiene un crédito de consumo.
* **Datos de contactos de la campaña actual**: Tipo de Comunicación (celular o teléfono fijo); Mes del último contacto; Día de la semana del contacto; duración de la llamada (segundos); Contacto: N° de contactos durante la campaña; DíasAtrás: días transcurridos desde último contacto; Resultado: resultado de la última llamada (falló, no-existe, éxito)
* **Datos socioeconómicos**: EmpTasaVar: tasa de variación de empleabilidad; IPC: índice de precios consumidor mensual; ICC: índice de confianza consumidor mensual; Euribor3m: tasa euribor de 3 meses indicador diario; NumEmpleados: cantidad de gente empleada, en indicador trimestral. 

Esta adaptación en particular, por el equipo de R:Solver (RSolver.com), enfrenta diferentes objetivos de aprendizaje dentro de los cursos de Big Data y Machine Learning.



## Instrucciones Generales
En este caso, se busca entender el comportamiento y desempeño de idferentes modelos de clasificación sobre este conjunto de datos, para predecir la variable de interés: **OK**, que servirá para predecir en casos futuros, según los datos de contactabilidad de un cliente, si el cliente aceptará o no contratar el depósito a plazo.

Todos los alumnos, ya sea en grupo, o individualmente (si no tienen compañeros), deben contestar las preguntas que se indican en las secciones de "Preguntas", más adelante. Se puede recurrir a ejercicios de otras fuentes, así como al material de clases.

El formato del email es:

To: rsandova@ing.puc.cl

Subject: Ejercicio 2 - (AÑO)(MES)(DÍA)    (por ej: Ejercicio 2 - 20200825)

Integrantes: (nombre y apellido), (nombre y apellido), ...

Tabla de datos de ejecuciones comparadas.

El mejor desempeño de modelo es (indicando análisis y sustento para la conclusión):



## Paso 0: Instalar las librerías de modelos de clasificación

Esto se ejecuta sólo una vez al comienzo de la sesión de cada persona. No se necesita volver a ejecutar con cada nueva prueba del resto de los scripts.

In [ ]:
install.packages('e1071')
install.packages('caret')
install.packages('rpart')
install.packages('rpart.plot')
install.packages('randomForest')
install.packages("nnet")


## Paso 1: Carga de los datos

La siguiente celda de código, carga los datos desde la URL de origen y luego muestra un encabezado con las primeras filas del dataset, para demostrar la disponibilidad de los datos.

Esto también puede ser ejecutado una sola vez, si es conveniente.

In [ ]:
# Se declara la URL de dónde obtener los datos
theUrlMain <- "http://www.rsolver.com/dcc/docs/bank-additional-full.csv"

# Se declaran los nombres de las columnas
columnas = c("Edad","Ocupación","EstadoCivil","Educación","Default","Hipotecario","Consumo","Contacto","Mes","Día",
             "Duración","NumContactos","DíasAtrás","Previo","ResultadoPrevio", "EmpTasaVar", "IPC", "ICC", "Euribor3m", "NumEmpleados", "OK")

# Se cargan datos principales a una estructura (marketing.data), asignando nombres de atributos a las columnas
marketing.data <- read.table(file = theUrlMain, header = TRUE, sep = ";", col.names = columnas, na.strings=c("unknown","NA"))

# Se eliminan los registros que tienen algún NA (unknown)
marketing.clean <- na.omit(marketing.data)
dim(marketing.clean) # Sólo quedan poco más de 30.000 filas (de las 41.000 originales)

# Se muestran las primeras líneas del dataset, incluyendo nombres asignados a las columnas
head(marketing.clean, 50)

# Se muestra un resumen de las columnas del dataset, describiendo los datos que se encuentran
summary(marketing.clean)


## Ejercicio - parte 1: Preparación conjuntos de entrenamiento y evaluación

El siguiente código prepara los datos para el entrenamiento y evaluación. Se pueden probar diferentes proporciones de entrenamiento vs. evaluación.


In [ ]:
# Proporción Training/Testing. Por ej: 0.75 = 75% entrenamiento y 25% validación
ratio = sample(1:nrow(marketing.clean), size = 0.75*nrow(marketing.clean)) 
Train = marketing.clean[ratio,] 
Test = marketing.clean[-ratio,] 
dim(Train)
dim(Test)

## Ejercicio - parte 2 complementaria: Implementación de un modelo de clasificación para comparar

Esta sección implementa un Random Forest, como modelo de clasificación de referencia, cuyo desempeño será comparado con la Red Neuronal más adelante.

**No se necesita modificar esta sección**


**Random Forest**

In [ ]:
library(randomForest)
library(caret)

# Random Forest
RF_model <- randomForest(OK ~ ., data=Train, importance=TRUE, keep.forest=TRUE)
RF_predict <- predict(RF_model, Test)
confusionMatrix(Test$OK, RF_predict)

# Ejercicio - parte 3 - Implementación de una Red Neuronal

A continuación se declara, entrena y evalúa un modelo de Red Neuronal. Esta primera declaración viene con una configuración inicial:

*   Se usa una capa escondida o intermedia. Su cantidad de nodos está dada por el atributo 'size'.

*   La cantidad de iteraciones para mejorar el entrenamiento se da por el atributo 'maxit'
*   Hay otros atributos posibles de analizar y modificar en https://www.rdocumentation.org/packages/nnet/versions/7.3-14/topics/nnet. Nótese que la configuración por defecto usa una activación logística, pero es posible aplicar softmax o linout, pero eso requiere de parámetros adicionales.

**NOTA**: dada la naturaleza aleatoria del comportamiento del entrenamiento, en ocasiones la red neuronal no entrega resultados para la clase menos representada. En cuyo caso, sólo basta con volver a ejecutar el código, para que - aleatoriamente - logre dar resultados en dicha clase.

**Ejercicio**
Los alumnos, en grupo o individualmente, deben lograr:

1.   De los datos de la tabla, obtener accurracy del modelo, recall de la clase positiva, ambos para poder evaluar su desempeño.
2.   Volver a ejecutar el modelo, cambiando la cantidad de nodos de la capa escondida (size) y aumentando (o disminuyendo) la cantidad de iteraciones (maxit).
3.   Opcionalmente, buscar algún otro parámetro de ejecución de la red neuronal, que pueda redundar en mejor desempeño.

Al terminar las ejecuciones, se debe enviar un email reportando los resultados y conclusiones, según las indicaciones al comienzo de este enunciado. La idea es contestar:

*    ¿Cuáles son los parámetros de ejecución del modelo que dan el mejor desempeño de la Red Neuronal?
*    ¿Con qué proporción de entrenamiento/validación se logra ese mejor desempeño?
*    ¿Logra superar al modelo Random Forest? (En cualquier caso, se pide una posible y teórica explicación de por qué es mejor/peor que ese otro modelo)



In [ ]:
library(nnet) 
library(caret)


# Red Neuronal
NN_model <- nnet(OK ~ ., data=Train, size=20, maxit=10)
NN_predict <- predict(NN_model, Test, type="class")

predTable <- table(NN_predict, Test$OK)
predTable

#Recordar que dada la naturaleza aleatoria del comportamiento del entrenamiento, 
# en ocasiones la red neuronal no entrega resultados para la clase menos representada. 
# En cuyo caso, sólo basta con volver a ejecutar el código, para que - aleatoriamente 
# - logre dar resultados en dicha clase.

tp = predTable[1,1]
fn = predTable[1,2]
fp = predTable[2,1]
tn = predTable[2,2]

accuracy2 = (tp+tn)/(tp+tn+fp+fn)
cat("\nAccuracy: ", accuracy2)

sensitivity2 = tp/(tp+fn)
cat("\nSensitivity: ", sensitivity2)

specificity2 = tn/(tn+fn)
cat("\nSpecificity: ", specificity2)

plot(predTable)
